In [212]:
from PIL import Image
from queue import Queue

import numpy as np
import time

In [226]:
def distance(x, y):
#     return (y[0]-x[0])**2 + (y[1]-x[1])**2 + (y[2]-x[2])**2
    return abs(y[0]-x[0]) + abs(y[1]-x[1]) + abs(y[2]-x[2])

In [181]:
class Pixel:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [182]:
def get_neighbors(p, width, height):
    neighbors = []
    for i in range(-1, 2):
        for j in range(-1, 2):
            if(not(i==0 and j==0)):
                xn = p.x + i
                yn = p.y + j
                if(xn>=0 and xn<width and yn>=0 and yn<height):
                    neighbors.append(Pixel(xn, yn))
                    
    return neighbors

In [202]:
def label_neighbors(fg_chk, q, width, height, current_label, labels):
    while(not q.empty()):
        p = q.get()
        for pn in get_neighbors(p, width, height):
            if(fg_chk[pn.y, pn.x]==1 and labels[pn.y, pn.x]==-1):
                labels[pn.y, pn.x] = current_label
                q.put(pn)
                
    return labels

In [243]:
def item_segmentation(image_no):    
    im_bg = Image.open("raw_images/bg.png")
    im_1 = Image.open(f"raw_images/{image_no}.png")
    
    (left, upper, right, lower) = (503, 263, 1436, 823)
    im_c_bg = im_bg.crop((left, upper, right, lower))
    im_c_1 = im_1.crop((left, upper, right, lower))

    im_c_bg.save("proc_images/bg_cropped.png")
    im_c_1.save(f"proc_images/im_{image_no}_cropped.png")
    
    width = right - left      # 933
    height = lower - upper    # 560

    im_sub = Image.new(mode='1', size=(width, height))

    start_time = time.time()
    
    for y in range(height):
        for x in range(width):
            p1 = im_c_bg.getpixel((x, y))
            p2 = im_c_1.getpixel((x, y))
            if(distance(p1, p2) > 30):
                im_sub.putpixel((x, y), 1)
                
    end_time = time.time()

    im_sub.save(f"proc_images/im_{image_no}_bg_subtraction.png")
    
    fg_chk = np.full((height, width), -1)
    labels = np.full((height, width), -1)
    
    for y in range(height):
        for x in range(width):
            fg_chk[y, x] = im_sub.getpixel((x, y))
        
    # Connected Component Labeling
    current_label = 0
    q = Queue()

    for y in range(height):
        for x in range(width):
            if(fg_chk[y, x]==1 and labels[y, x]==-1):
                labels[y, x] = current_label
                q.put(Pixel(x, y))
                labels = label_neighbors(fg_chk, q, width, height, current_label, labels)
                current_label += 1
    
    im_label = Image.new(mode='1', size=(width, height))
    item_im_arr = []

    for i in range(current_label):
        x_min = width
        x_max = 0
        y_min = height
        y_max = 0

        for y in range(height):
            for x in range(width):
                if(labels[y, x]==i):
                    if(x < x_min):
                        x_min = x
                    if(x > x_max):
                        x_max = x
                    if(y < y_min):
                        y_min = y
                    if(y > y_max):
                        y_max = y

        item_im_arr.append(im_c_1.crop((x_min, y_min, x_max, y_max)))

        for y in range(y_min, y_max+1):
            for x in range(x_min, x_max+1):
                im_label.putpixel((x, y), 1)

    im_label.save(f"proc_images/im_{image_no}_item_segmentation.png")
    
    cnt = 0
    for im in item_im_arr:
        cnt += 1
        filename = f"proc_images/im_{image_no}_item_{cnt}.png"
        im.save(filename)

#     print(end_time - start_time)
        
    return item_im_arr

In [250]:
for image_no in range(1, 11):
    print(f"image no. {image_no}")
    item_im_arr = item_segmentation(image_no)
    
    for im in item_im_arr:
        red_cnt = 0
        for y in range(im.height):
            for x in range(im.width):            
                p = im.getpixel((x, y))
                if(p[0]>160 and p[1]<90 and p[2]<90):
                    red_cnt += 1

        print(red_cnt)
        print(red_cnt/(im.height*im.width))
    print("===================")

image no. 1
77
0.012349639133921411
476
0.5118279569892473
image no. 2
471
0.5233333333333333
88
0.01199400299850075
image no. 3
84
0.01347233360064154
476
0.5118279569892473
image no. 4
472
0.5244444444444445
243
0.006763903579580248
19
0.029320987654320986
image no. 5
736
0.007737594617325484
472
0.5075268817204301
image no. 6
405
0.0033320992225101815
11
0.016975308641975308
0
0.0
465
0.5166666666666667
image no. 7
357
0.003299689440993789
11
0.016975308641975308
0
0.0
465
0.5166666666666667
image no. 8
0
0.0
220
0.010671840892553965
316
0.33978494623655914
image no. 9
196
0.005316840277777778
472
0.5075268817204301
0
0.0
84
0.011437908496732025
image no. 10
128
0.02043096568236233
107
0.020272830617658204
19
0.029320987654320986
468
0.5032258064516129
